In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

file_path = 'data/tgv_data.csv'

df = pd.read_csv(file_path, sep=';')
df.head()

In [ ]:
df.info()

In [ ]:
# Supprimer les colonnes vides ou partiellement remplies
df = df.drop(columns=['commentaire_annulation', 'commentaire_retards_depart', 'commentaires_retard_arrivee'])

In [ ]:
df.describe()

## Missing dates

In [ ]:
df.groupby(['gare_depart', 'gare_arrivee']).size()
df.groupby(['gare_depart', 'gare_arrivee']).size().reset_index(name='count').query('count < 66')


### Paris Vaugirard train station

In [ ]:
df_vaugirard = df[(df['gare_arrivee'] == 'PARIS VAUGIRARD') | (df['gare_depart'] == 'PARIS VAUGIRARD')]
df_vaugirard.groupby(['gare_depart', 'gare_arrivee']).size()

In [ ]:
df_vaugirard_nantes = df[((df['gare_depart'] == 'PARIS VAUGIRARD') | (df['gare_depart'] == 'PARIS MONTPARNASSE')) & (df['gare_arrivee'] == 'BORDEAUX ST JEAN') & (df['date'] == '2018-01')]
df_vaugirard_nantes

Tous les trajets qui impliquent la gare PARIS VAUGIRARD n'ont lieu qu'en 2018. Plus aucun trajet ensuite. On peut donc supprimer ces lignes du df.

In [ ]:
df = df[(df['gare_depart'] != 'PARIS VAUGIRARD') & (df['gare_arrivee'] != 'PARIS VAUGIRARD')]

In [ ]:
df.groupby(['gare_depart', 'gare_arrivee']).size().reset_index(name='count').query('count < 66')
# df_with_data.groupby(['gare_depart', 'gare_arrivee']).size().reset_index(name='count')

### Dealing with Tourcoing - Marseille, Madrid - Marseille, Marseille - Madrid

In [ ]:
df_tourcoingmarseille = df[(df['gare_depart'] == 'TOURCOING') & (df['gare_arrivee'] == 'MARSEILLE ST CHARLES')]

# Filter the DataFrame to keep rows where the trip is not from TOURCOING to MARSEILLE ST CHARLES
df = df[~((df['gare_depart'] == 'TOURCOING') & (df['gare_arrivee'] == 'MARSEILLE ST CHARLES'))]


In [ ]:
df_madrid_marseille = df[((df['gare_depart'] == 'MADRID') & (df['gare_arrivee'] == 'MARSEILLE ST CHARLES')) 
                         | ((df['gare_depart'] == 'MARSEILLE ST CHARLES') & (df['gare_arrivee'] == 'MADRID'))]
df_madrid_marseille

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Liste des trajets d'intérêt
trajets = [
    ('MADRID', 'MARSEILLE ST CHARLES'),
    ('MARSEILLE ST CHARLES', 'MADRID')
]

# Filtrer le DataFrame pour les trajets spécifiques en utilisant isin()
filtered_df = df[df[['gare_depart', 'gare_arrivee']].apply(tuple, axis=1).isin(trajets)]

# Convertir la colonne 'date' en type datetime si ce n'est pas déjà le cas
filtered_df['date'] = pd.to_datetime(filtered_df['date'], format='%Y-%m')

# Grouper par paire de gares et date, puis calculer le retard moyen d'arrivée
grouped = filtered_df.groupby(['gare_depart', 'gare_arrivee', 'date'])['retard_moyen_arrivee'].mean()

# Créer un graphique pour chaque trajet
plt.figure(figsize=(12, 6))

for trajet, data in grouped.groupby(['gare_depart', 'gare_arrivee']):
    plt.plot(data.index.get_level_values('date'), data.values, marker='o', linestyle='-', label=trajet)

plt.title('Évolution du Retard Moyen d\'Arrivée par Trajet')
plt.xlabel('Date')
plt.ylabel('Retard Moyen d\'Arrivée')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
df = df[~((df['gare_depart'] == 'MADRID') & (df['gare_arrivee'] == 'MARSEILLE ST CHARLES') | ((df['gare_depart'] == 'MARSEILLE ST CHARLES') & (df['gare_arrivee'] == 'MADRID')))]

In [ ]:
df.groupby(['gare_depart', 'gare_arrivee']).size().reset_index(name='count').query('count < 66')

In [ ]:
import pandas as pd

# Obtenez la liste des combinaisons uniques de gares de départ et d'arrivée dans le DataFrame
trajets = df[['gare_depart', 'gare_arrivee']].drop_duplicates()

# Liste des dates attendues (de janvier 2018 à juin 2023, en incluant tous les mois)
dates_attendues = [f"{year}-{month:02d}" for year in range(2018, 2023) for month in range(1, 13)]
dates_attendues += ["2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06"]

# Parcourez les combinaisons de gares de départ et d'arrivée existantes
for index, trajet in trajets.iterrows():
    gare_depart = trajet['gare_depart']
    gare_arrivee = trajet['gare_arrivee']
    
    for date in dates_attendues:
        # Vérifiez si la combinaison de gare de départ, gare d'arrivée et date n'existe pas dans le DataFrame
        if not df[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'] == date)].empty:
            continue

        # Traitez la date manquante en utilisant le code que vous avez fourni
        previous_dates = [d for d in dates_attendues if d < date][:3]
        next_dates = [d for d in dates_attendues if d > date][:3]

        previous_rows = df[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & df['date'].isin(previous_dates)]
        next_rows = df[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & df['date'].isin(next_dates)]

        mean_values = (previous_rows.mean() + next_rows.mean()) / 2
        mean_values['date'] = date
        service = df.loc[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'] == previous_dates[0]), 'service'].values[0]

        new_row = {col: mean_values[col] for col in mean_values.index}
        new_row['service'] = service
        new_row['gare_depart'] = gare_depart
        new_row['gare_arrivee'] = gare_arrivee

        # Ajoutez la nouvelle ligne au DataFrame
        df = df.append(new_row, ignore_index=True)

        # Log : Signaler l'ajout de la nouvelle ligne
        print(f"Nouvelle ligne ajoutée : {new_row}")


### Traiter les mois où aucun train n'a circulé (COVID)

In [ ]:
zero_train = df[(df['nb_train_prevu'] == 0)]
zero_train

In [ ]:
zero_train = df[(df['nb_train_prevu'] == 0)]
zero_train.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Liste des trajets d'intérêt
trajets = [
    ('NANTES', 'STRASBOURG')
]

# Filtrer le DataFrame pour les trajets spécifiques en utilisant isin()
filtered_df = df[df[['gare_depart', 'gare_arrivee']].apply(tuple, axis=1).isin(trajets)]

# Convertir la colonne 'date' en type datetime si ce n'est pas déjà le cas
filtered_df['date'] = pd.to_datetime(filtered_df['date'], format='%Y-%m')

# Grouper par paire de gares et date, puis calculer le retard moyen d'arrivée
grouped = filtered_df.groupby(['gare_depart', 'gare_arrivee', 'date'])['retard_moyen_arrivee'].mean()

# Créer un graphique pour chaque trajet
plt.figure(figsize=(12, 6))

for trajet, data in grouped.groupby(['gare_depart', 'gare_arrivee']):
    plt.plot(data.index.get_level_values('date'), data.values, marker='o', linestyle='-', label=trajet)

plt.title('Évolution du Retard Moyen d\'Arrivée par Trajet')
plt.xlabel('Date')
plt.ylabel('Retard Moyen d\'Arrivée')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

# Compteur pour suivre le nombre de lignes modifiées
nb_lignes_modifiees = 0

# Obtenez la liste des combinaisons uniques de gares de départ et d'arrivée dans le DataFrame
trajets = df[['gare_depart', 'gare_arrivee']].drop_duplicates()
# Liste des dates attendues (de janvier 2018 à juin 2023, en incluant tous les mois)
dates_attendues = [f"{year}-{month:02d}" for year in range(2018, 2023) for month in range(1, 13)]
dates_attendues += ["2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06"]

# Colonnes numériques à mettre à jour
update_columns = ['duree_moyenne', 'nb_train_prevu', 'nb_annulation', 'nb_train_depart_retard', 'retard_moyen_depart', 'retard_moyen_tous_trains_depart',
                  'nb_train_retard_arrivee', 'retard_moyen_arrivee', 'retard_moyen_tous_trains_arrivee', 'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
                  'nb_train_retard_sup_30', 'nb_train_retard_sup_60', 'prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
                  'prct_cause_materiel_roulant', 'prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs']

for index, trajet in trajets.iterrows():
    gare_depart = trajet['gare_depart']
    gare_arrivee = trajet['gare_arrivee']
    
    for date in dates_attendues:

        # Vérifiez si 'nb_train_prevu' est égal à zéro pour la date actuelle
        if df[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'] == date)]['nb_train_prevu'].values[0] == 0:
            # Calculez les nouvelles valeurs pour les colonnes numériques
            other_years_data = df[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'].str[-2:] == date[-2:]) & (df['date'].str[:4] != date[:4])]
            if not other_years_data.empty:
                mean_values = other_years_data.mean()
                mean_values['date'] = date

                # Obtenir le service existant
                service = df.loc[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'] == date), 'service'].values[0]
                mean_values['service'] = service

                # Mise à jour des colonnes numériques
                for col in update_columns:
                    df.loc[(df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee) & (df['date'] == date), col] = mean_values[col]
                # Incrémentez le compteur de lignes modifiées
                nb_lignes_modifiees += 1

# Affichez le nombre total de lignes modifiées
print(f"Nombre total de lignes modifiées : {nb_lignes_modifiees}")
# Affichez le DataFrame mis à jour


### Detection des valeurs erronnées

In [ ]:
import pandas as pd

# Liste des colonnes commençant par "prct_"
prct_columns = [col for col in df.columns if col.startswith("prct_")]

# Créez un DataFrame pour stocker les lignes qui ne respectent pas le critère
invalid_rows = pd.DataFrame()

# Vérifiez que la somme des colonnes prct_ n'est pas proche de 100 pour chaque ligne
for index, row in df.iterrows():
    sum_prct = sum(row[prct_columns])
    if abs(sum_prct - 100) > 2:  # Vous pouvez ajuster la tolérance en changeant la valeur 2
        # Créez un dictionnaire avec les valeurs à inclure dans le DataFrame
        invalid_data = {
            'date': row['date'],
            'gare_depart': row['gare_depart'],
            'gare_arrivee': row['gare_arrivee']
        }
        invalid_data.update({col: row[col] for col in prct_columns})
        
        # Ajoutez ces données au DataFrame invalid_rows
        invalid_rows = invalid_rows.append(invalid_data, ignore_index=True)

# Affichez les lignes qui ne respectent pas le critère
print("Lignes ne respectant pas le critère de somme égale à 100 :")
invalid_rows


### Outliers

In [ ]:
df.describe()

In [ ]:
import pandas as pd

# Your DataFrame (df) and the column name with outliers
column_name = 'retard_moyen_arrivee'
n = 3  # Number of previous and next values to consider for the mean

# Step 1: Detect rows with outliers (negative values)
outliers = df[df[column_name] < 0].sort_values(by=column_name, ascending=True)

# Initialize a counter for the number of lines modified
num_lines_modified = 0

# Step 2: Calculate the mean of the 3 previous and 3 next values for each row with outliers
for index, row in outliers.iterrows():
    date = row['date']
    gare_depart = row['gare_depart']
    gare_arrivee = row['gare_arrivee']
    # Find the index of the row with the same date in the original DataFrame
    original_index = df[(df['date'] == date) & (df['gare_depart'] == gare_depart) & (df['gare_arrivee'] == gare_arrivee)].index[0]
    
    nb_train_retard_sup_15 = row['nb_train_retard_sup_15']
    retard_moyen_trains_retard_sup15 = row['retard_moyen_trains_retard_sup15']
    nb_train_retard_sup_30 = row['nb_train_retard_sup_30']
    nb_train_retard_sup_60 = row['nb_train_retard_sup_60']

    new_value = (nb_train_retard_sup_15 * retard_moyen_trains_retard_sup15 + 
                 nb_train_retard_sup_30 * 30 + 
                 nb_train_retard_sup_60 * 60) / (nb_train_retard_sup_15 + nb_train_retard_sup_30 + nb_train_retard_sup_60)
    
    # Replace the value in the DataFrame
    df.at[original_index, column_name] = new_value
    
    # Increment the counter for modified lines
    num_lines_modified += 1

# Print the number of lines modified
print(f"Number of lines modified: {num_lines_modified}")

## CReation nouvelle feature - durée strandard

In [ ]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df

# Calculate the difference 'duree_moyenne - retard_moyen_tous_trains_arrivee'
df['difference'] = df['duree_moyenne'] - df['retard_moyen_tous_trains_arrivee']

# Calculate the median of the 'difference' column for each combination of 'gare_depart' and 'gare_arrivee'
df['duree_standard'] = df.groupby(['gare_depart', 'gare_arrivee'])['difference'].transform('median')

# Round the 'duree_standard' column to the nearest integer using NumPy
df['duree_standard'] = np.round(df['duree_standard'])

# Drop the 'difference' column if you no longer need it
df.drop(columns=['difference'], inplace=True)

# Save the new dataset

In [ ]:
# Sauvegarder le DataFrame prétraité dans un fichier CSV
df.to_csv("data/preprocessed_tgv_data.csv", index=False, sep=";")